In [1]:
#%load_ext autoreload
#%autoreload 2

In [1]:
from model import MLP
from optimizers import SGLD, H_SA_SGHMC
from mnist_utils import load_mnist_dataset
from trainer import BNNTrainer
import torch
from torch.nn import functional as F

import numpy as np
import dill as pickle
from pathlib import Path

In [2]:
batch_size = 500
input_dim = 784
width = 100
depth = 2
output_dim = 10
lr = 1e-3
n_epoch = 100
alpha0, beta0 = 10, 10
resample_prior_every = 15
resample_momentum_every = 50
burn_in_epochs = 10
save_freq = 2
resample_prior_until = 50

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
trainloader, valloader = load_mnist_dataset('data', batch_size)

In [4]:
model = MLP(input_dim=input_dim, width=width, depth=depth, output_dim=output_dim)

In [5]:
#optimizer = SGLD(model.parameters(), lr=lr, alpha0=alpha0, beta0=beta0)
optimizer = H_SA_SGHMC(model.parameters(), lr=lr, alpha0=alpha0, beta0=beta0)
#optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=False)

In [6]:
def nll_func(y_hat, y):
    nll = F.cross_entropy(y_hat, y, reduction='sum')
    return nll

def err_func(y_hat, y):
    err = y_hat.argmax(-1).ne(y)
    return err

In [7]:
trainer = BNNTrainer(model, optimizer, nll_func, err_func, trainloader, valloader, device=device, 
    resample_prior_every=resample_prior_every,
    resample_momentum_every=resample_momentum_every,
    save_freq=save_freq,
    batch_size=batch_size
    )

In [8]:
trainer.train(n_epoch=n_epoch, burn_in_epochs=burn_in_epochs, resample_prior_until=resample_prior_until)

2020-08-14 10:33:00,364 Epoch 0 finished. Val loss 0.16795885562896729, Val error 0.0522
2020-08-14 10:33:13,539 Epoch 1 finished. Val loss 0.14774039387702942, Val error 0.0449
2020-08-14 10:33:23,898 Epoch 2 finished. Val loss 0.15454192459583282, Val error 0.0468
2020-08-14 10:33:34,645 Epoch 3 finished. Val loss 0.13956117630004883, Val error 0.0395
2020-08-14 10:33:43,440 Epoch 4 finished. Val loss 0.14896883070468903, Val error 0.0391
2020-08-14 10:33:51,539 Epoch 5 finished. Val loss 0.1541183441877365, Val error 0.0404
2020-08-14 10:34:00,327 Epoch 6 finished. Val loss 0.14277158677577972, Val error 0.0378
2020-08-14 10:34:07,655 Epoch 7 finished. Val loss 0.16034924983978271, Val error 0.0391
2020-08-14 10:34:14,958 Epoch 8 finished. Val loss 0.19193580746650696, Val error 0.0474
2020-08-14 10:34:22,457 Epoch 9 finished. Val loss 0.1724758744239807, Val error 0.0404
2020-08-14 10:34:30,700 Epoch 10 finished. Val loss 0.1634906530380249, Val error 0.0395
2020-08-14 10:34:39,268

In [34]:
opt_with_priors = trainer.optimizer

priors = {}
group_params = opt_with_priors.param_groups[0]['params']
for (n, _), p in zip(model.named_parameters(), group_params):  
    state = opt_with_priors.state[p]  
    priors[n] = state['weight_decay']

In [10]:
weights_set = trainer.weight_set_samples[-(n_epoch - resample_prior_until) // 2:]
pickle.dump(weights_set, Path('weights.pkl').open('wb'))

def state_dict_to_vec(state_dict):
    return torch.cat([w_i.view(-1) for w_i in state_dict.values()])

squeezed_weights = [state_dict_to_vec(w) for w in weights_set]

models = [MLP(input_dim=input_dim, width=width, depth=depth, output_dim=output_dim) for w in weights_set]
for w, model in zip(weights_set, models):
    model.load_state_dict(w)

In [14]:
squeezed_weights[-1].shape

torch.Size([89610])

In [ ]:
def get_prediction(x, model):
    return F.softmax(model(x), dim=-1)

def get_binary_prediction(x, model, classes):
    assert len(classes) == 2
    return F.softmax(model(x)[..., classes], dim=-1)

In [ ]:
def compute_log_likelihood(x, y, model):
    assert len(classes) = 2
    y_hat = model(x)
    log_likelihood = -F.cross_entropy(y_hat, y, reduction='mean')

    return log_likelihood

def compute_mc_estimate(function: callable, models, x: torch.tensor):
    res = 0.0
    for model in models:
        res += function(x, model)
    return res / len(models)

def compute_naive_variance(function:callable, control_variate: callable, models, x: torch.tensor):
    return torch.sum(torch.tensor([(function(x, model) - control_variate(x, model))**2 for model in models])) / (len(models) - 1)

In [ ]:
def compute_tricky_divergence(model):
    div = 0
    for p in model.parameters():
        if p.grad is not None:
            div += p.grad.sum()

    return div

def stein_control_variate(psy_function, train_x, train_y, model, opt_with_priors, N_train, new_x):
    log_likelihood = compute_log_likelihood(train_x, train_y, model) * N_train
    log_likelihood.backward()
    
    ncv_value = 0
    psy_value = psy_function(state_dict_to_vec(model.state_dict()), new_x)
    psy_value.backward()

    for n, p in model.named_parameters():
        d_p = p.grad.data
        d_p.add_(p, alpha=-priors[n])

        psy_value += 




    weight = state_dict_to_vec(model.state_dict())
    phi_weigth = phi_function(weight, x)
    phi_weight.backward()
    
    control_variate = 

def wan_control_variate(psy_function, model, x):

In [15]:
from torch import nn

class PsyLinear(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.layer = nn.Linear(input_dim, 1)

    def forward(self, weights, x):
        return  self.layer(weights)

class PsyMLP(nn.Module):
    def __init__(self, input_dim, width, depth):
        super().__init__()

        self.input_dim = input_dim
        self.width = width
        self.depth = depth

        layers = [nn.Linear(input_dim, width), nn.ReLU()]
        for i in range(depth - 1):
            layers.append(nn.Linear(width, width))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(width, 1))

        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)


In [16]:
psy_linear = PsyLinear(squeezed_weights[0].shape[0])

psy_linear.to(device)

PsyLinear(
  (layer): Linear(in_features=89610, out_features=1, bias=True)
)

In [19]:
out = psy_linear(squeezed_weights[0], None)

In [20]:
out

tensor([-0.1204], grad_fn=<AddBackward0>)

In [41]:
out.backward()